In [5]:
import cv2 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf

from PIL import Image, ImageFilter


In [6]:
# importing the dataset
path = r"C:\Users\CC\Machine_learning_projects\Resistor Color Code recognition app\Dataset"
os.path.exists(path)

True

In [7]:
# load data
data = tf.keras.utils.image_dataset_from_directory('./Dataset')

Found 2857 files belonging to 37 classes.


In [8]:
# 91 batches
len(data)

90

In [9]:
type(data)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [10]:
data_iter =data.as_numpy_iterator()
batch = data_iter.next()

In [11]:
# a batch contains 32 images and 32 labels
len(batch)

2

In [12]:
type(data_iter)

tensorflow.python.data.ops.dataset_ops._NumpyIterator

In [13]:
# number of images in 1 batch is 32
len(batch[0])

32

In [14]:
batch[0].shape

(32, 256, 256, 3)

In [15]:
# preprocessing 
data = data.map(lambda x, y: (x/255.0, y))

In [16]:
data.as_numpy_iterator().next()[0].max()

1.0

In [17]:
# splitting the data

train_size = int(len(data)* .7)
val_size = int(len(data)*.1)
test_size = int(len(data)*.2)



In [18]:
train = data.take(train_size) 
test = data.skip(train_size).take(test_size)
val = data.skip(train_size + test_size).take(val_size)

In [19]:
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input,Flatten,Dropout,Dense, BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam 
from keras.regularizers import l1, l2
from sklearn.metrics import confusion_matrix

In [20]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (2,2), 1, activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(rate=0.25), #adding dropout regularization throughout the model to deal with overfitting
    # The second convolution
    tf.keras.layers.Conv2D(32, (2,2), 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(rate=0.1),
    # The third convolution
    tf.keras.layers.Conv2D(64, (2,2), 1,  activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(rate=0.10),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(64, activation='relu'),

    # 3 output neuron for the 3 classes of Animal Images
    tf.keras.layers.Dense(13, activation='softmax')
    ])



In [27]:
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='SparseCategoricalCrossentropy',optimizer="adam",metrics=['acc'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 255, 255, 16)      208       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 127, 127, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 63, 63, 32)        0

In [29]:
logdir = r"C:\Users\CC\Machine_learning_projects\Resistor Color Code recognition app\logs"

In [30]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

In [31]:
hist = model.fit(train, epochs = 25, validation_data = val, callbacks = [tensorboard_callback])

Epoch 1/25


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\users\cc\appdata\local\programs\python\python39\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\users\cc\appdata\local\programs\python\python39\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\CC\myvirtualenv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\CC\myvirtualenv\lib\site-packages\traitlets\config\application.py", line 965, in launch_instance
      app.start()
    File "C:\Users\CC\myvirtualenv\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\CC\myvirtualenv\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_

In [106]:
for folders in os.listdir(path):
    for files in os.listdir(os.path.join(path,folders)):
        filename = files +'.jpg'
        print(filename)
        break
    break    

100R_1-4W_(1).jpg.jpg


In [5]:
count = 0
path2 = r'C:\Users\CC\Machine_learning_projects\Resistor Color Code recognition app\Customized dataset'
os.chdir(path2)
for folders in os.listdir(path):
    for files in os.listdir(os.path.join(path,folders)):
        image = cv2.imread(os.path.join(path,folders,files))
        #print(image)
        #plt.imshow(image)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.medianBlur(image, 5)
        
        #image = image/255.0
        
        #parent_dir = folders
        #direc = os.mkdir(os.path.join(path2,parent_dir))
        
       
        filename = folders + '.jpg'
        cv2.imwrite(filename,image)
        break
    break    
       

In [115]:
image.max()

0.7803921568627451

In [80]:
# load data
data = tf.keras.utils.image_dataset_from_directory(path)

Found 2881 files belonging to 37 classes.


In [81]:
type(data)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [82]:
data_iter = data.as_numpy_iterator()

In [83]:
batch = data_iter.next()

In [85]:
batch[0][0].max()

253.2901